In [1]:
import numpy as np
import glob #this package allows for the searching of files and outputs their filepaths
from numpy import char

In [11]:
#This script will generate a .txt file of the names of all image cubes
ListoNames=glob.glob('/Volumes/REU_2023/Per-emb-8_Models/Extension_Work/IREAndKeplerian/*9.78.fits') #Grab filepaths for all image cubes
                                    #Change the filepath above. Make not of the numbers here^
                                    #As this is what will ultimatly make sure we select image cubes, not PV diagrams
                                    #Change to the relevant number for your model batch, in addition to the directorypath

#This section cleans up the glob.glob output
for i in range(len(ListoNames)):
    if ListoNames[i][0:14] == 'cubechi2/fits/':
        ListoNames[i] = ListoNames[i][14:]
    elif ListoNames[i][0:9] == 'cubechi2/':
        ListoNames[i] = ListoNames[i][9:]
ListoNames=np.array(ListoNames) #Send it to an np.array
for i in range(len(ListoNames)):
    ListoNames[i]='\n'+ListoNames[i] #Adding a newline character to separate the filepaths out
    if ListoNames[i][-1] != 's': #Fixing the missing 's' in 'fits' in some files
        temp = '' #The missing s problem is not fully fixed on my machine, so I have to go in manually myself anyways. But maybe it works for you
        temp = ListoNames[i]+'s\n'
        ListoNames[i]=temp
        
        
file=open('ListofNames.txt', 'w') #Making a new .txt file to write to
for i in range(len(ListoNames)): #Writing all the filepaths with their newline characters to the file
    file.writelines(ListoNames[i])
file.close() #And done


#EDIT FILE TO REMOVE FIRST EMPTY LINE
#ALSO, FIND AND REPLACE ALL .fit WITH .fits BUT KEEP THE NEWLINE
#Also remove filepath from all files, should just be the actual name of the fits file, no directory
#It's just be a find and replace job on a text editor, find all the line from start to last / and delete

In [9]:
def TemplateGroup(ModelFilename, ModelFilepath, ObservedFilename, ObservedFilepath, Noise, Cutoff, ObservingArea=np.zeros(5), OffsetVelocity = 0, Weight=0):
    '''
    This function will write out a complete 'chunk' for the input file for cubechi2, using filenames already made
    This is to bypass the issues seen with cubechi2 making filenames already generated, and messing that up
    
    Inputs
    -----------------------
    ModelFilename: Single string, filename of the FERIA model cube
    ModelFilepath: Single string, take the filepath to the files, starting after /Volumes/. Should end with the last /
    ObservedFilename: Single string, observed data being compared to the model.
    ObservedFilepath: Single string, observed data filepath starting after /Volumes/. End on / like ModelFilename
    OffsetVelocity: Float m/s, How much to adjust the model before comparison for systemic velocity. Should for our case be 0 so defaults(edit: LEAVE IT 0 AT ALL TIMES)
    Noise: float Jy/beam, RMS noise level
    Cutoff: Threshold to cutoff data below noise level
    ObservingArea: 1D array 6 entries, RA, Dec, Velocity min and max values
    Weight: Weighting on chi2 calculation, will default to 0(which tells cubechi to figure it out), but setting it manually here is an option also
    
    Output
    -----------------------
    8 Line text, properly formatted such that concatination keeps the input file valid. Simply rename to .in, use in cubechi2
    '''
    
    ModelFilenameEdit = ModelFilename[:-6]     #removes .fits from filename
    line1 = ModelFilepath + ModelFilenameEdit + '.out\n' 
    line2 = ObservedFilepath + ObservedFilename + '\n'
    line3 = ModelFilepath + ModelFilename
    line4 = str(OffsetVelocity)+'    # Offset for Systemic Velocity (m/s)\n'
    line5 = str(Noise) + ' ' + str(Cutoff)+'   # rms (Jy/beam), threshold (Jy/beam)\n'
    line6 = str(ObservingArea[0]) + ' ' +  str(ObservingArea[1]) + ' '+ str(ObservingArea[2]) + ' ' + str(ObservingArea[3]) + ' ' + str(ObservingArea[4]) + ' ' + str(ObservingArea[5]) +  '    # Ramin, Ramax, Decmin, Decmax, Vmin, Vmax\n'
    line7 = str(Weight) + '    # Weight // If this value is <=0, weight is calculated\n' 
    line8 = '\n'
    CombinedParagraph = line1+line2+line3+line4+line5+line6+line7+line8
    return CombinedParagraph
    

In [12]:
#This script will let you read in the list of filenames, and compile that into a input file form
names=open('ListofNames.txt') #Make sure you have done any manual work to this file
file=open('Per8ExtensionBlended.txt', 'w') #Change filename to what you will use
testsetup=names.readlines()
for i in range(len(testsetup)):
    paragraph = TemplateGroup(testsetup[i], #Model cube names
                              'REU_2023/Per-emb-8_Models/Extension_Work/IREAndKeplerian/', #filepath to model cube
                              'Per-emb-8_vel_subcube_centercut.fits', #name of the observed data
                              'REU_2023/Per-emb-8_Models/', #filepath of the observed data #NO /VOLUMES/
                              .005, #noise
                              4*.005, #threshold
                              [0,360,-180,180,7933.26,13224.9]) #coordinate stuffs
                                                #^these 2^ numbers should change, VMin and VMax in Data Logging Spreadsheet
    file.writelines(paragraph)
file.close()

#Somehow it makes the last entry have the systemic velocity not newline, go to last paragraph and manually hit enter separating the lines. 
#The first blank line after a paragraph should be on a multiple of 8